In [1]:
import numpy as np

x = np.zeros((40,1000))

In [3]:
x.shape

(40, 1000)

In [12]:
y = x[:,0:100]

In [13]:
y.shape

(40, 100)

In [5]:
'''
python sticky_snippet_net.py mode model_file data_folder
'''

import os
import sys
import tensorflow as tf
import numpy as np
import string

#global variables
mode_fun = None
model_file = None
data_folder = None
dir_path = os.path.dirname(os.path.realpath('__file__'))
count_nonstick = 0
count_12 = 0
count_34 = 0
count_56 = 0
count_78 = 0
count_stick = 0
learning_rate = 0.5

one_hot_dict = {
    "12-STICKY": 0,
    "34-STICKY": 1,
    "56-STICKY": 2,
    "78-STICKY": 3,
    "STICK_PALINDROME": 4,
    "NONSTICK": 5,
}

def matches(a, b):
    if (a == 'A' and b == 'C') or (a == 'C' and b == 'A'):
        return 1
    if (a == 'B' and b == 'D') or (a == 'D' and b == 'B'):
        return 1
    return 0


def get_label(dna):
    global count_12, count_34, count_56, count_78, count_nonstick, count_stick
    k = 0
    if not matches(dna[k],dna[-1]):
        count_nonstick += 1
        return "NONSTICK"

    while matches(dna[k],dna[-k-1]) and k < 20:
        k += 1

    if k == 1 or k == 2:
        count_12 += 1
        return "12-STICKY"
    if k == 3 or k == 4:
        count_34 += 1
        return "34-STICKY"
    if k == 5 or k == 6:
        count_56 += 1
        return "56-STICKY"
    if (k == 7 or k == 8) and k < 20 :
        count_78 += 1
        return "78-STICKY"
    count_stick += 1
    return "STICK_PALINDROME"


def one_hot_encoding(label):
    y_data = np.zeros((6,), dtype=np.int)
    y_data[one_hot_dict[label]] = 1
    return y_data

In [6]:
def translate(x):
    inp = "ABCD"
    out = "1234"
    s = string.translate(x, string.maketrans(inp,out))
    x_vec = [float(i) for i in s]
    # print "x_vec: ",x_vec
    return x_vec

In [20]:
def read_data():
    x = []
    y = []
    folder_path = os.path.join(dir_path, data_folder)
    if not os.path.isdir(folder_path):
        raise ValueError('FOLDER NOT FOUND')
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if not files:
        raise ValueError('NO DATA')
    for f in files:
        filename = os.path.join(folder_path, f)
        # Check if file is a text file
        if filename.endswith('.txt'):
            filedata = open(filename,'r')
        else:
            continue

        for line in filedata:
            line = line.strip('\n')
            if len(line) != 40:
                continue
            # print line
            x_data = translate(line)
            # print "x_data: ",x_data
            # break
            y_data = one_hot_encoding(get_label(line))
            x.append(x_data)
            y.append(y_data)

    return np.transpose(x), np.transpose(y)

In [8]:
def perceptron(w_name, w_shape, input, b_name, b_shape, init):
    W = tf.get_variable(w_name, shape=w_shape, initializer=init)
    b = tf.get_variable(b_name, shape=b_shape, initializer=init)
    # print "x: ",input.shape, " W: ", w_shape, " b: ", b_shape
    return tf.nn.relu(tf.matmul(W, input) + b)

In [9]:
def nn():

    init = tf.contrib.layers.xavier_initializer()
    x = tf.placeholder(dtype=tf.float32, shape=(40, None))
    y = tf.placeholder(dtype=tf.float32, shape=(6, None))
    # layer 1
    y1 = perceptron("W1", (6, 40), x, "b1", (6, 1), init)

    # layer 2
    y2 = perceptron("W2", (6, 6), y1, "b2", (6, 1), init)

    # layer 3
    y3 = perceptron("W3", (6, 6), y2, "b3", (6, 1), init)

    # layer 4
    y4 = perceptron("W4", (6, 6), y3, "b4", (6, 1), init)

    # layer 5 (output)
    # W = tf.get_variable("W3", shape=(6, 10), initializer=init)
    # b = tf.get_variable("b3", shape=(10, 1), initializer=init)
    # y5 = tf.matmul(W, y4) + b

    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y4))  ##reduce_sum
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)
    return train_step, x, y

In [10]:
def training():
    x_data, x_label = read_data()
    total = np.shape(x_data)[0]
    train, x, y = nn()
    print "x: ", np.shape(x_data)
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    for i in range(0, total, 1000):
        end = i+1000
        batch = x_data[:, i:end]
        y_ = x_label[:, i:end]
        print "batch: ", np.shape(batch)
        session.run(train,feed_dict={x: batch, y: y_}) #feed_dict=
    saver = tf.train.Saver()
    saver.save(session, model_file)
    return 0

In [11]:
def fivefold_training():
    return 0


def testing():
    return 0

In [12]:
mode = {
    "train": training,
    "5fold": fivefold_training,
    "test": testing
}


def print_help():
    print "python sticky_snippet_net.py mode model_file data_folder"


def parse_args():
    print sys.argv
    global mode_fun, model_file, data_folder
    mode_fun = mode[sys.argv[1]]
    model_file = str(sys.argv[2])
    data_folder = str(sys.argv[3])

In [13]:
# read_data()
# mode_fun()



In [21]:
model_file = "model_file"
data_folder = 'data'

x_data, x_label = read_data()

In [22]:
total = np.shape(x_data)[0]

In [23]:
train, x, y = nn()
print "x: ", np.shape(x_data)
session = tf.Session()
session.run(tf.global_variables_initializer())

x:  (40, 10000)


In [27]:
for i in range(0, total, 1000):
    end = i+1000
    batch = x_data[:, i:end]
    y_ = x_label[:, i:end]
    print "batch: ", np.shape(batch)
    session.run(train,feed_dict={x: batch, y: y_}) #feed_dict=
# saver = tf.train.Saver()
# saver.save(session, model_file)

batch:  (40, 1000)


In [4]:
os.path.dirname(os.path.realpath('__file__'))

'/Users/kushal/Study/AI Chris Pollet/Assignment 3'

In [25]:
batch = x_data[:, 0:1000]
y_ = x_label[:, 0:1000]
print "batch: ", np.shape(batch), " y_: ", np.shape(y_)

batch:  (40, 1000)  y_:  (6, 1000)


In [26]:

session.run(train,feed_dict={x: batch, y: y_})